In [1]:
import os
from docx import Document
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import openai

In [2]:
# Укажите путь к файлу / chemin d'accès au fichier
downloads_path = os.path.expanduser("Desktop")  # Путь к папке "Загрузки"
file_name = "Bat-ADAPT.docx"  # Название файла
file_path = os.path.join(downloads_path, file_name)

In [3]:
# Функция для чтения текста из .docx
# Fonction de lecture de texte à partir de .docx
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for paragraph in doc.paragraphs:
        if paragraph.text.strip():  # Пропускаем пустые строки / Sauter les lignes vides
            full_text.append(paragraph.text.strip())
    return " ".join(full_text)

In [4]:
document_text = read_docx(file_path)
print("Contenu du document:")
print(document_text[:500])  # Выводим первые 500 символов / Nous affichons les 500 premiers caractères

Contenu du document:
Exemples de Questions et Réponses pouvant alimenter le Chatbox de Bat-ADAPT Périmètre géographique des analyses Bat-ADAPT Question : Dans quels pays en Europe peut-on utiliser cet outil ? Est-ce possible au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque ? Réponse : L'outil Bat-ADAPT permet des analyses de risques climatiques des bâtiments situés en Europe Continentale. Il est tout à fait possible d'utiliser Bat-ADAPT pour les bâtiments situés au Royaume-Uni, en Suisse, en Pologne ou


In [5]:
# Добавьте вывод всех параграфов документа, чтобы увидеть, как python-docx извлекает текст:
# Ajouter une impression de tous les paragraphes du document pour voir comment python-docx extrait le texte :

def debug_document(file_path):
    doc = Document(file_path)
    print("Contenu du document:")
    for i, paragraph in enumerate(doc.paragraphs):
        print(f"Paragraphe {i + 1}: '{paragraph.text}'")

# Путь к вашему Word-файлу
debug_document(file_path)

Contenu du document:
Paragraphe 1: 'Exemples de Questions et Réponses pouvant alimenter le Chatbox de Bat-ADAPT'
Paragraphe 2: ''
Paragraphe 3: 'Périmètre géographique des analyses Bat-ADAPT '
Paragraphe 4: ''
Paragraphe 5: 'Question : Dans quels pays en Europe peut-on utiliser cet outil ? Est-ce possible au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque ?'
Paragraphe 6: ''
Paragraphe 7: 'Réponse : L'outil Bat-ADAPT permet des analyses de risques climatiques des bâtiments situés en Europe Continentale. Il est tout à fait possible d'utiliser Bat-ADAPT pour les bâtiments situés au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque. '
Paragraphe 8: ''
Paragraphe 9: 'Gestion de la base de données de bâtiments '
Paragraphe 10: ''
Paragraphe 11: 'Question : Quand nous ajoutons un nouveau fichier Excel dans notre espace "Mon Patrimoine" alors qu’un précédent Excel avait été importé, est-ce que cela va écraser les précédentes données ?'
Paragraphe 12: ''
Paragraphe 13: 'R

In [6]:
# Извлечение текста из документа Word
# Extraire du texte d'un document Word

def extract_questions_answers(file_path):
    """
    Читает вопросы и ответы из файла .docx.
    Возвращает список пар (вопрос, ответ).

    Lit les questions et les réponses à partir d'un fichier .docx.
    Renvoie une liste de paires (question, réponse).
    
    """
    doc = Document(file_path)
    qa_pairs = []
    current_question = None

    # Проходим по каждому абзацу
    # Nous parcourons chaque paragraphe
    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        
        # Убираем неразрывные пробелы и приводим к стандартному виду
        # Supprimer les espaces insécables et ramener au format standard
        text = text.replace('\xa0', ' ').strip()

        # Проверяем на вопрос
        # Vérifiez la question
        if text.lower().startswith("question :"):
            current_question = text.replace("Question :", "").strip()
        
        # Проверяем на ответ
        # Vérifier la réponse
        elif text.lower().startswith("réponse :") and current_question:
            answer = text.replace("Réponse :", "").strip()
            qa_pairs.append({"question": current_question, "answer": answer})
            current_question = None
    
    return qa_pairs

In [7]:
qa_pairs = extract_questions_answers(file_path)
if not qa_pairs:
    print("Erreur : Aucune question ou réponse trouvée dans le document.")
else:
    print(f"{len(qa_pairs)} paires de questions et réponses trouvées:")
    for pair in qa_pairs:
        print(f"Question: {pair['question']}")
        print(f"Réponses: {pair['answer']}\n")

10 paires de questions et réponses trouvées:
Question: Dans quels pays en Europe peut-on utiliser cet outil ? Est-ce possible au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque ?
Réponses: L'outil Bat-ADAPT permet des analyses de risques climatiques des bâtiments situés en Europe Continentale. Il est tout à fait possible d'utiliser Bat-ADAPT pour les bâtiments situés au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque.

Question: Quand nous ajoutons un nouveau fichier Excel dans notre espace "Mon Patrimoine" alors qu’un précédent Excel avait été importé, est-ce que cela va écraser les précédentes données ?
Réponses: Oui, les données des bâtiments ayant le même "Code de bâtiment" seront écrasées par celles du nouveau fichier Excel importé.

Question: Comment peut-on supprimer un bâtiment, notamment les bâtiments qui ont servi de test ?
Réponses: Pour supprimer des bâtiments, il faut les sélectionner dans la liste en cochant leurs cases.

Question: Est-il possible 

In [8]:
# Векторизация текста и создание индекса
# Vectorisation de texte et création d'index

def create_vector_index(question_pairs, model_name="sentence-transformers/all-MiniLM-L6-v2"):
    model = SentenceTransformer(model_name)
    questions = [qa['question'] for qa in question_pairs]
    embeddings = model.encode(questions, convert_to_tensor=False)
    dimension = embeddings[0].shape[0]
    
    # Создаем индекс FAISS
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))
    return index, model, question_pairs


In [9]:
# Создание индекса
# Créer un index
index, model, qa_pairs = create_vector_index(qa_pairs)

In [10]:
# Поиск ответа по запросу
# Rechercher une réponse à une demande

def search_answer(query, index, model, qa_pairs, top_k=1):
    query_embedding = model.encode([query], convert_to_tensor=False)
    distances, indices = index.search(np.array(query_embedding), top_k)
    results = [{"question": qa_pairs[idx]["question"], "answer": qa_pairs[idx]["answer"], "distance": dist}
               for dist, idx in zip(distances[0], indices[0])]
    return results

In [11]:
# Le Chatbot
def chat_bot(qa_pairs, index, model):
    print("Le chatbot est prêt à fonctionner! Entrez votre question (ou 'exit' pour sortir):")
    while True:
        user_query = input("Vous: ")
        if user_query.lower() == 'exit':
            print("Au revoir!")
            break
        results = search_answer(user_query, index, model, qa_pairs, top_k=1)
        if results:
            print(f"Le chatbot: {results[0]['answer']} (question: {results[0]['question']})")
        else:
            print("Le chatbot: Désolé, je n'ai pas trouvé de réponse appropriée.")


In [ ]:
# Запуск
# Lancement
if __name__ == "__main__":
    
    # Извлечение вопросов и ответов
    # Extraction des questions et des réponses
    qa_pairs = extract_questions_answers(file_path)
    
    # Создание индекса
    # Créer un index
    index, model, qa_pairs = create_vector_index(qa_pairs)
    
    # Запуск чат-бота
    # Lancer un chatbot
    chat_bot(qa_pairs, index, model)

Le chatbot est prêt à fonctionner! Entrez votre question (ou 'exit' pour sortir):


Vous:  Comment peut-on supprimer un bâtiment, notamment les bâtiments qui ont servi de test ?


Le chatbot: Pour supprimer des bâtiments, il faut les sélectionner dans la liste en cochant leurs cases. (question: Comment peut-on supprimer un bâtiment, notamment les bâtiments qui ont servi de test ?)


Vous:  Quand nous ajoutons un nouveau fichier Excel dans notre espace "Mon Patrimoine" alors qu’un précédent Excel avait été importé, est-ce que cela va écraser les précédentes données ?


Le chatbot: Oui, les données des bâtiments ayant le même "Code de bâtiment" seront écrasées par celles du nouveau fichier Excel importé. (question: Quand nous ajoutons un nouveau fichier Excel dans notre espace "Mon Patrimoine" alors qu’un précédent Excel avait été importé, est-ce que cela va écraser les précédentes données ?)
